In [1]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import numpy as np
import math
import datetime
connection_uri = (
    "mssql+pyodbc://TEDPHO0134\TANDT_DP/DP_Tan?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri)

<h4>1. Lấy bảng PREMIUM <h4>

In [2]:
cutoff = '20230131'
premium_query = f"select * from PREMIUM_KPI where CUTOFF = '{cutoff}'"
premium = pd.read_sql(premium_query, conn)

In [3]:
premium_table = premium

<h4>1.1 Lấy bảng DAILYSALES <h4>



In [ ]:
dailysales_query = f"select * from KPI_DAILYSALES where CUTOFF = '{cutoff}'"
dailysales = pd.read_sql(dailysales_query, conn)

<h4> 2. Lấy ra PERIOD<h4>

In [10]:
premium_table = premium_table.loc[:, premium_table.columns != 'Link_Key']

In [12]:
#Thêm cột ngày xét
Monthly_Premium = premium_table
Monthly_Premium['Date_On_Set'] = "" #Tạo ra cột trống
Monthly_Premium['Date_On_Set']= Monthly_Premium['Date_On_Set'].str.cat(Monthly_Premium['CUTOFF'].str[:4] ,sep ="")
Monthly_Premium['Date_On_Set']= Monthly_Premium['Date_On_Set'].str.cat(Monthly_Premium['CUTOFF'].str[4:6] ,sep ="-")
Monthly_Premium['Date_On_Set']= Monthly_Premium['Date_On_Set'].str.cat(Monthly_Premium['CUTOFF'].str[6:8] ,sep ="-")

In [13]:
#Phải biến thành kiểu datetime trước
Monthly_Premium['Date_On_Set'] = pd.to_datetime(Monthly_Premium['Date_On_Set'])
#Thêm cột ngày đầu tháng:
#-> Đưa vào ngày 1 của tháng
#LƯU Ý CÁC NGÀY OFF ĐẶC BIỆT CỦA THÁNG 4.2023 VÀ 01/2024
if not('20240229' in Monthly_Premium['CUTOFF'].values): 
    Monthly_Premium['Start_Month'] = (Monthly_Premium['Date_On_Set'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))
elif ('20230531' in Monthly_Premium['CUTOFF'].values):
    Monthly_Premium.loc[:,'Start_Month'] = '2023-04-29'
else:
    Monthly_Premium.loc[:,'Start_Month'] = '2024-01-27'

In [16]:
Monthly_Premium.columns

Index(['Policy No', 'Product Code', 'Premium transaction', 'PREM TRAN NAME',
       'Premium Collected', 'Collected Date', 'Applied Premium Date',
       'FYP Before Discount', 'FYP Discount', 'FYP', 'RYP', 'Topup Premium',
       'Premium Term', 'Premium Year', 'Policy Status', 'Policy Year',
       'Policy Term', 'Frequency of Payment', 'Issued Date', 'Effected Date',
       'Terminated Date', 'Lapse Date', 'Due date', 'Next Due Date',
       'Transfer Date', 'POLICY ACKNOWLED', 'Sum Assure', 'Area Code',
       'Servicing Agent', 'Freelook', 'Proposal Receive Date',
       'RISK_COMMENCE_DATE', 'Age_Customer', 'Issuing Agent', 'AFYP', 'CUTOFF',
       'RETURN_DATE', 'Last_Policy_STT', 'Date_On_Set', 'Start_Month'],
      dtype='object')

In [17]:
#Lấy ra ngày Period:
#LƯU Ý CÁCH CỘNG 1 NGÀY VÀO CỘT APPLIED PREMIUM DATE
Period_Com = Monthly_Premium
Period_Com ['Date_Period_1'] = np.where( (Period_Com['Policy Status'] == 'FL'), Period_Com['Issued Date'],
np.where( (Period_Com['POLICY ACKNOWLED'].isnull()), Period_Com['Issued Date'],
np.where((((Period_Com['Applied Premium Date'] + datetime.timedelta(days=1)) - Period_Com['POLICY ACKNOWLED']) / np.timedelta64(1, 'D')) < 22,
Period_Com['Issued Date'], Period_Com['Applied Premium Date'])))

In [20]:
Period_Prem = Period_Com
calculate_condition = ((Period_Prem['Applied Premium Date'] >= Period_Prem['Start_Month']) & (Period_Prem['Applied Premium Date'] <= Period_Prem['Date_On_Set']))
PREM_month = Period_Prem[((Period_Prem['Applied Premium Date'] >= Period_Prem['Start_Month']) & (Period_Prem['Applied Premium Date'] <= Period_Prem['Date_On_Set']))]

<h4>3. Tính CaseCount <h4>

In [27]:
CC_prem = PREM_month
CC_prem['Status Transition'] = CC_prem['Last_Policy_STT'] + "to" + CC_prem['Policy Status'] 

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_17644\632978849.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CC_prem['Status Transition'] = CC_prem['Last_Policy_STT'] + "to" + CC_prem['Policy Status']


In [ ]:
#Loại bỏ khoản trắng:


In [51]:
CC_prem['PREM TRAN NAME'] = CC_prem['PREM TRAN NAME'].str.strip()

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_17644\1807299906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CC_prem['PREM TRAN NAME'] = CC_prem['PREM TRAN NAME'].str.strip()


In [54]:
CC_prem['Count_Stage_1'] = np.where((CC_prem['PREM TRAN NAME'] == 'Contract Issue') & (CC_prem['Last_Policy_STT'].isnull()) , 1,
                        np.where((CC_prem['PREM TRAN NAME'] == 'Freelook Cancellation') & (CC_prem['Last_Policy_STT'].isnull()),-1, 0))

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_17644\3210236533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CC_prem['Count_Stage_1'] = np.where((CC_prem['PREM TRAN NAME'] == 'Contract Issue') & (CC_prem['Last_Policy_STT'].isnull()) , 1,


In [59]:
CC_prem['Count_Stage_2'] = np.where((CC_prem['Status Transition'].str.len() >5) &\
                        CC_prem['Status Transition'].str[4:].isin(['FL', 'CF', 'DC', 'DH', 'NT', 'PO', 'WD']), -1,
                        np.where((CC_prem['Status Transition'].str.len() < 5) &\
                        CC_prem['Status Transition'].str[4:].isin(['IF']), 1, 0))

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_17644\3233848328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CC_prem['Count_Stage_2'] = np.where((CC_prem['Status Transition'].str.len() >5) &\


In [61]:
CC_prem['Count_Policy'] = np.where((CC_prem['Count_Stage_2']) == 0, CC_prem['Count_Stage_1'], CC_prem['Count_Stage_2'] )

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_17644\3545740601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CC_prem['Count_Policy'] = np.where((CC_prem['Count_Stage_2']) == 0, CC_prem['Count_Stage_1'], CC_prem['Count_Stage_2'] )


<h4>4. Sum để lấy CaseCount <h4>

In [67]:
#Nếu muốn check trạng thái CC của HD, check chỗ này
CaseCount = CC_prem
CaseCount = CaseCount[['Issuing Agent', 'Policy No', 'CUTOFF', 'Count_Policy']].drop_duplicates()
Sum_CC = CaseCount.groupby(['Issuing Agent', 'Policy No', 'CUTOFF'])['Count_Policy'].sum()
Sum_CC = Sum_CC.reset_index()

In [73]:
Sum_CC.loc[(Sum_CC['Count_Policy'] <0),'CC(-)'] = Sum_CC['Count_Policy']
Sum_CC.loc[(Sum_CC['Count_Policy'] >0),'CC(+)'] = Sum_CC['Count_Policy']

In [76]:
Sum_CC

,Issuing Agent,Policy No,CUTOFF,Count_Policy,CC(-),CC(+)
0,60000073,80000089,20230131,0,NaN,NaN
1,60000156,80000302,20230131,0,NaN,NaN
2,60000204,80000483,20230131,0,NaN,NaN
3,60000482,80132262,20230131,1,NaN,1.0
4,60000717,80132247,20230131,1,NaN,1.0
...,...,...,...,...,...,...
4400,69999993,80000222,20230131,0,NaN,NaN
4401,69999996,80000077,20230131,0,NaN,NaN
4402,69999997,80072097,20230131,0,NaN,NaN
4403,69999997,80073594,20230131,0,NaN,NaN


<h5>4.1 Sum lại CaseCount theo Đại lý phát hành HD <h4>

In [78]:
Agent_CC = Sum_CC.groupby(['Issuing Agent', 'CUTOFF'])[['CC(-)', 'CC(+)']].sum()
Agent_CC = Agent_CC.reset_index()

<h4>5. Tính FYP <h4>

In [113]:
FYP_month = PREM_month
FYP_month = pd.merge(FYP_month, Sum_CC, left_on = 'Policy No', right_on = 'Policy No', how = 'left' )
FYP_month = FYP_month[['Policy No', 'Product Code', 'Premium transaction', 'PREM TRAN NAME',
       'Premium Collected', 'Collected Date', 'Applied Premium Date',
       'FYP Before Discount', 'FYP Discount', 'FYP', 'RYP', 'Topup Premium',
       'Premium Term', 'Premium Year', 'Policy Status', 'Policy Year',
       'Policy Term', 'Frequency of Payment', 'Issued Date', 'Effected Date',
       'Terminated Date', 'Lapse Date', 'Due date', 'Next Due Date',
       'Transfer Date', 'POLICY ACKNOWLED', 'Sum Assure', 'Area Code',
       'Servicing Agent', 'Freelook', 'Proposal Receive Date',
       'RISK_COMMENCE_DATE', 'Age_Customer', 'Issuing Agent_x', 'AFYP',
       'CUTOFF_x', 'RETURN_DATE', 'Count_Policy_y']].rename(columns={'Issuing Agent_x': 'Issuing Agent', 'Count_Policy_y': 'Count_Policy','CUTOFF_x':'CUTOFF'})

<h5> 5.1 Sum lại theo Servicing Agent lấy FYP <h5>

In [116]:
Sum_FYP = FYP_month
Sum_FYP['FYP_Total'] = Sum_FYP['FYP'] + Sum_FYP['Topup Premium']*0.1
Sum_FYP.loc[(Sum_FYP['FYP_Total'] <0), 'FYP(-)'] = Sum_FYP['FYP_Total']
Sum_FYP.loc[(Sum_FYP['FYP_Total'] >0), 'FYP(+)'] = Sum_FYP['FYP_Total']
Sum_FYP = Sum_FYP.groupby(['Servicing Agent','Policy No', 'CUTOFF', 'Count_Policy'])['FYP_Total'].sum()
Sum_FYP = Sum_FYP.reset_index()
#Rules: Các HD Mark -1=> HD giảm phí, tính FYP(-), các HD đang còn IF, trạng thái 0, 1 => FYP(+)
Sum_FYP.loc[(Sum_FYP['Count_Policy'] <0), 'FYP(-)'] = Sum_FYP['FYP_Total']
Sum_FYP.loc[(Sum_FYP['Count_Policy'] >=0), 'FYP(+)'] = Sum_FYP['FYP_Total']
Sum_FYP = Sum_FYP.groupby(['Servicing Agent', 'CUTOFF'])[['FYP(-)', 'FYP(+)']].sum()
Sum_FYP

,,FYP(-),FYP(+)
Servicing Agent,CUTOFF,,
60000194,20230131,0.0,0.0
60000449,20230131,0.0,0.0
60000482,20230131,0.0,16026000.0
60000564,20230131,0.0,0.0
60000717,20230131,0.0,16141000.0
...,...,...,...
60076140,20230131,0.0,10003000.0
60076141,20230131,0.0,30256000.0
60076148,20230131,0.0,32048000.0


In [117]:
Sum_FYP.to_csv('checkFYP.csv')